### Imports & constants

In [1]:
import os
import glob

import numpy as np
import matplotlib.pyplot as plt

import utils

In [2]:
HNSCC_PATH = os.path.join(os.curdir, 'datasets', 'HNSCC')
OUTPUT_PATH = os.path.join(os.curdir, 'output', 'HNSCC.npz')

### Data processing

In [3]:
imgs_s = [] # np.array with sagittal MIPs
imgs_f = [] # np.array with frontal MIPs
ids = [] # np.array with CTs identifiers
thicks = [] # np.array with CTs slices' thicknesses

In [4]:
subdirs = glob.glob(os.path.join(HNSCC_PATH, '*', '*', '*'))
subdirs = [subdir for subdir in subdirs if len(os.listdir(subdir)) > 1]

In [5]:
for i, subdir in enumerate(subdirs):
    # --- loading CT ---
    try:
        img, spacings, slice_thickness = utils.read_dicom(subdir)
    except ValueError as e:
        print(f'An exception for {subdir} occured: {str(e)}')
        continue
    
    # --- preprocessing CT ---
    # converting to HU scale
    img = utils.to_HU(img)
    
    # mass center calculation + 0ing everything outside body
    img, center_h, center_w = utils.get_mass_center(img)
    
    # frontal MIP
    img_cropped = utils.crop_ct(img, center_h, center_w)
    img_mip_f = np.amax(img_cropped, axis=0)
    img_mip_f = np.swapaxes(img_mip_f, 0, 1)
    
    # rescaling to 1mm x 1mm
    img_mip_f = utils.rescale_mip(img_mip_f, spacings[0], slice_thickness)
    
    # sagittal MIP
    img_cropped = utils.crop_ct(img, center_h, center_w, for_frontal=False)
    img_mip_s = np.amax(img_cropped, axis=1) 
    img_mip_s = np.swapaxes(img_mip_s, 0, 1)
    
    # rescaling to 1mm x 1mm
    img_mip_s = utils.rescale_mip(img_mip_s, spacings[0], slice_thickness)
    
    # saving
    imgs_s.append(img_mip_s)
    imgs_f.append(img_mip_f)
    ids.append(subdir.split('/')[4])
    thicks.append(slice_thickness)

In [6]:
# exporting to .npz
imgs_s = np.asarray(imgs_s)
imgs_f = np.asarray(imgs_f)
ids = np.asarray(ids)
thicks = np.asarray(thicks)
n_mips = imgs_s.shape[0]

np.savez_compressed(OUTPUT_PATH, imgs_s=imgs_s, imgs_f=imgs_f, ids=ids, thicks=thicks, n_mips=n_mips)

/home/dittohead/miniconda3/envs/thesis/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
